In [ ]:
from subprocess import Popen, PIPE, CalledProcessError
import re

def callPopen(cmd):
    print(cmd)
    with Popen(cmd.split(),
               stdout=PIPE,
               stderr=PIPE,
               bufsize=1,
               universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),
               stdout=PIPE,
               stderr=PIPE,
               bufsize=1,
               universal_newlines=True) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)
    

# Automated calibration for the GOTM/FABM model at Langtjern

## Pre-requisites

* gcloud command line utility
* google cloud account, you might need to be an admin for some of this to work, also this is not free
* ssh key setup in gitlab.au.dk
* fabric3 needs to be (pip) installed
* python modules listed in the first cell of this notebook

## Creating an instance on Google Cloud

In [ ]:
instanceName = 'gotm-cal'
username = 'jose-luis'

createInstance = '''\
gcloud compute instances create {} \
--zone europe-west3-a \
--image-family debian-9 \
--image-project debian-cloud \
--boot-disk-size 200GB \
--machine-type n1-standard-2 \
'''

deleteInstance = '''\
gcloud compute instances delete {} \
--zone europe-west-3a \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone europe-west3-a --metadata-from-file ssh-keys={}'''

keyDir = ('/home/jose-luis/.ssh/gotmKeys')

ip=''
instanceExists,ip = isInstance(instanceName)

print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone europe-west3-a'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    callPopen('rm -rf {}'.format(keyDir))
    callPopen('mkdir -p {}'.format(keyDir))
    p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(keyDir,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
             )
    print(p.communicate())
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    ip=isInstance(instanceName)[1]
        #callPopen('chmod 400 {}'.format(keyDir))
        
print("The ip of {} is {}".format(instanceName,ip))

## Updating fabfile.py with credentials and ip

In [ ]:
callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('stage',ip))

#Testing connection
#Adding key to remote machine
p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
print(p.communicate())

## Setting up GOTM and FABM in the remote machine

In [ ]:
callPopen('fab testConnection')
callPopen('fab getUtilities')
callPopen('fab downloadModels') #note that the location of the key file is hardcoded in the getFABM function
callPopen('fab compileModels')
callPopen('fab getPROGNOS')
callPopen('fab testRun:/home/{}/PROGNOS/langtjern/langtjern.xml'.format(username))